## Import Libraries

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
from tqdm import tqdm

## Import Model

In [2]:
from model import Net

## Print Model Summary

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
       BatchNorm2d-2           [-1, 10, 26, 26]              20
           Dropout-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             900
       BatchNorm2d-5           [-1, 10, 24, 24]              20
           Dropout-6           [-1, 10, 24, 24]               0
            Conv2d-7           [-1, 10, 22, 22]             900
       BatchNorm2d-8           [-1, 10, 22, 22]              20
           Dropout-9           [-1, 10, 22, 22]               0
           Conv2d-10           [-1, 10, 20, 20]             900
      BatchNorm2d-11           [-1, 10, 20, 20]              20
           Conv2d-12           [-1, 10, 20, 20]             100
      BatchNorm2d-13           [-1, 10, 20, 20]              20
        MaxPool2d-14           [-1, 10,

## Transformations for Training and Testing Datasets

In [4]:
# Train Dataset transformations
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop((28, 28), (0.85, 1.0)),
    transforms.RandomRotation((-7., 7.), fill=0),
    transforms.Normalize((0.1307,), (0.3081,)),
    ])

# Test Dataset transformations
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((28, 28)),
    transforms.Normalize((0.1325,), (0.3104,))
    ])

## Training and Testing Datasets

In [5]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

## Training and Testing Dataloaders

In [6]:
SEED = 8

cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)
if cuda:
    torch.cuda.manual_seed(SEED)

# Dataloader arguments
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=128)

# Train & Test Dataloaders
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? False


## Training and Testing Loops

In [7]:

def train(model, device, train_loader, optimizer, epoch):
    """
    Training Loop
    :param model: Model
    :param device: Device
    :param train_loader: Training Dataloader
    :param optimizer: Optimizer
    :param epoch: Epoch
    """
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'    Loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    """
    Testing Loop
    :param model: Model
    :param device: Device
    :param test_loader: Testing Dataloader
    """
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('    Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Model Training and Testing

In [8]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print(f"Epoch {epoch}:")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch 1:


    Loss=0.2085336595773697 batch_id=468: 100%|██████████| 469/469 [01:03<00:00,  7.43it/s] 


    Test set: Average loss: 0.0904, Accuracy: 9825/10000 (98.25%)

Epoch 2:


    Loss=0.12635397911071777 batch_id=468: 100%|██████████| 469/469 [00:59<00:00,  7.84it/s] 


    Test set: Average loss: 0.0552, Accuracy: 9873/10000 (98.73%)

Epoch 3:


...

    Test set: Average loss: 0.0256, Accuracy: 9916/10000 (99.16%)

Epoch 19:


    Loss=0.06821335107088089 batch_id=468: 100%|██████████| 469/469 [01:07<00:00,  6.98it/s] 


    Test set: Average loss: 0.0208, Accuracy: 9940/10000 (99.40%)

Epoch 20:


    Loss=0.06556873023509979 batch_id=468: 100%|██████████| 469/469 [01:06<00:00,  7.03it/s] 


    Test set: Average loss: 0.0239, Accuracy: 9938/10000 (99.38%)



Target achieved in the 19th epoch